# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [13]:
import requests
import pyowm
import json

import os
from dotenv import load_dotenv

load_dotenv()  

api_key = os.getenv("API_KEY")


## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [15]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key

}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 288.31,
  'feels_like': 287.53,
  'temp_min': 286.48,
  'temp_max': 290.36,
  'pressure': 1022,
  'humidity': 63,
  'sea_level': 1022,
  'grnd_level': 1018},
 'visibility': 10000,
 'wind': {'speed': 2.24, 'deg': 167, 'gust': 6.26},
 'clouds': {'all': 100},
 'dt': 1745584808,
 'sys': {'type': 2,
  'id': 2091269,
  'country': 'GB',
  'sunrise': 1745556218,
  'sunset': 1745608377},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [16]:
#your code here
from pyowm.owm import OWM
from pyowm.utils.config import get_default_config

config_dict = get_default_config()
config_dict['language'] = 'ru'  # your language here, eg. French
owm = OWM(api_key, config_dict)
mgr = owm.weather_manager()
observation = mgr.weather_at_place('Berlin, Germany')
print(observation.weather.detailed_status)


ясно


## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [17]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°K, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 288.31°K, Humidity: 63%, Wind Speed: 2.24 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [18]:
#your code here
observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather
weather.status           # short version of status (eg. 'Rain')
display(weather.detailed_status)
temp_dict_celsius = weather.temperature('celsius')
display(temp_dict_celsius)

'пасмурно'

{'temp': 15.46,
 'temp_max': 17.27,
 'temp_min': 13.89,
 'feels_like': 14.71,
 'temp_kf': None}

## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [19]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

display(geocode_response)

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

{'coord': {'lon': -74.006, 'lat': 40.7143},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 290.12,
  'feels_like': 289.47,
  'temp_min': 288.69,
  'temp_max': 291.25,
  'pressure': 1024,
  'humidity': 61,
  'sea_level': 1024,
  'grnd_level': 1022},
 'visibility': 10000,
 'wind': {'speed': 3.6, 'deg': 190},
 'clouds': {'all': 0},
 'dt': 1745585316,
 'sys': {'type': 2,
  'id': 2037026,
  'country': 'US',
  'sunrise': 1745575330,
  'sunset': 1745624724},
 'timezone': -14400,
 'id': 5128581,
 'name': 'New York',
 'cod': 200}

Then, request the weather forecast for that latitude/longitude:

In [20]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-04-25 15:00:00, Temp: 291.33°C
Time: 2025-04-25 18:00:00, Temp: 294.08°C
Time: 2025-04-25 21:00:00, Temp: 295.65°C
Time: 2025-04-26 00:00:00, Temp: 292.06°C
Time: 2025-04-26 03:00:00, Temp: 289.62°C
Time: 2025-04-26 06:00:00, Temp: 287.26°C
Time: 2025-04-26 09:00:00, Temp: 287.92°C
Time: 2025-04-26 12:00:00, Temp: 288.5°C
Time: 2025-04-26 15:00:00, Temp: 291.09°C
Time: 2025-04-26 18:00:00, Temp: 292.76°C
Time: 2025-04-26 21:00:00, Temp: 294.34°C
Time: 2025-04-27 00:00:00, Temp: 291.14°C
Time: 2025-04-27 03:00:00, Temp: 285.14°C
Time: 2025-04-27 06:00:00, Temp: 283.19°C
Time: 2025-04-27 09:00:00, Temp: 282.46°C
Time: 2025-04-27 12:00:00, Temp: 282.71°C
Time: 2025-04-27 15:00:00, Temp: 284.25°C
Time: 2025-04-27 18:00:00, Temp: 284.26°C
Time: 2025-04-27 21:00:00, Temp: 285.23°C
Time: 2025-04-28 00:00:00, Temp: 286.31°C
Time: 2025-04-28 03:00:00, Temp: 285.97°C
Time: 2025-04-28 06:00:00, Temp: 285.11°C
Time: 2025-04-28 09:00:00, Temp: 283.88°C
Time: 2025-04-28 12:00:00, Temp: 28

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [21]:
#your code here
from pyowm.utils import timestamps
import pprint

mgr = owm.weather_manager()
h3_forecast = mgr.forecast_at_place('Berlin,DE', '3h')
tomorrow_at_five = timestamps.tomorrow(17, 0)                      # datetime object for tomorrow at 5 PM
weather = h3_forecast.get_weather_at(tomorrow_at_five) 
weather_data = weather.to_dict()
weather_data


{'reference_time': 1745690400,
 'sunset_time': None,
 'sunrise_time': None,
 'clouds': 15,
 'rain': {},
 'snow': {},
 'wind': {'speed': 3.97, 'deg': 57, 'gust': 5.91},
 'humidity': 39,
 'pressure': {'press': 1027, 'sea_level': 1027},
 'temperature': {'temp': 285.51,
  'temp_kf': 0,
  'temp_max': 285.51,
  'temp_min': 285.51,
  'feels_like': 283.82},
 'status': 'Clouds',
 'detailed_status': 'небольшая облачность',
 'weather_code': 801,
 'weather_icon_name': '02d',
 'visibility_distance': 10000,
 'dewpoint': None,
 'humidex': None,
 'heat_index': None,
 'utc_offset': None,
 'uvi': None,
 'precipitation_probability': 0}

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [22]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 288.31°C, Fahrenheit: 550.958°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [23]:
#your code here
observation = mgr.weather_at_place('Dubai')  # the observation object is a box containing a weather object
weather = observation.weather
weather.status           # short version of status (eg. 'Rain')

temp_dict_celsius = weather.temperature('celsius')
temp_dict_farenheit = weather.temperature('fahrenheit')
print(f"temp {temp_dict_celsius['temp']}°C, Fahrenheit: {temp_dict_farenheit['temp']}°F")

temp 31.96°C, Fahrenheit: 89.53°F
